Roosevelt University

https://www.roosevelt.edu/contact/directory?txtSearch=&ddlGroupSearch%5B%5D=1&ddlCollegeSearch=All&ddlInstrumentSearch=All&ddlDepartmentSearch=678

**Libraries**

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

**Initializing an instance of Google Chrome.**

In [2]:
# Initialize function
service = Service()

# Set preferences for Chrome
options = webdriver.ChromeOptions()

# Start
driver = webdriver.Chrome(service=service, options=options)


In [3]:
url = "https://www.roosevelt.edu/contact/directory?txtSearch=&ddlGroupSearch%5B%5D=1&ddlCollegeSearch=All&ddlInstrumentSearch=All&ddlDepartmentSearch=678"
driver.get(url)

driver.implicitly_wait(10)  # Wait 10 seconds 

In [4]:
# Get the HTML of the page after it has loaded completely
html = driver.page_source

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, "html.parser")


**Finding the professors' names with BS.**

In [5]:
# Find all divs with class "indFacultyListing-infoContainer"
divs_professors = soup.find_all("div", class_="indFacultyListing-infoContainer")

# List to store the data of professors
data_professors = []

In [18]:
data_professors = []

for div_professor in divs_professors:

    #COLLEGE
    college_prefessor = "Rosevelt University"

    #NAME
    nome_professor = div_professor.find("div", class_="indDirectoryListing-name").find("span").get_text(strip=True)
    
    #LINK
    link_professor = div_professor.find("a", title="View {}'s Faculty Profile".format(nome_professor.strip())).get("href")
    
    # Get the full URL 
    full_link_professor = "https://www.roosevelt.edu" + link_professor
    
    # Open the professor's profile page with Selenium
    driver.get(full_link_professor)
    
    # Wait for the page to load completely
    driver.implicitly_wait(10)
    
    # Get the HTML of the profile page after it has loaded completely
    profile_html = driver.page_source
    
    # Parse the HTML using BeautifulSoup
    profile_soup = BeautifulSoup(profile_html, "html.parser")

    try:
        # COURSE
        course_name_span = profile_soup.find("span", id="collegeName")
        course_prefessor = course_name_span.find_next_sibling("div").get_text(strip=True)
    except AttributeError:
        course_prefessor = "NaN"

    try:
        # EMAIL
        email_prefessor = profile_soup.find('div', class_='field_email').find('a').text.strip()
    except AttributeError:
        email_prefessor = "NaN"

    try:
        # RESEARCH AREA
        research_area_elem = profile_soup.find('ul', class_='facultyDescription-sectionList').find('li')
        
        # Extrair o texto da área de pesquisa primária
        research_professor = research_area_elem.get_text(strip=True)
    except AttributeError:
        research_professor = "NaN"
        
    data_professors.append({
        "College": college_prefessor,
        "Name": nome_professor,
        "Link": full_link_professor,
        "Course": course_prefessor,
        "Email": email_prefessor,
        "Research Area": research_professor,
    })

df_professors = pd.DataFrame(data_professors)

print(df_professors)


               College                    Name  \
0  Rosevelt University          Evgeny Dantsin   
1  Rosevelt University  Saba Jamaliannasrabadi   
2  Rosevelt University            Michael Ruth   
3  Rosevelt University           Rami Salahieh   
4  Rosevelt University            Alex Wolpert   
5  Rosevelt University              Ray Wright   

                                                Link  \
0         https://www.roosevelt.edu/profile/edantsin   
1  https://www.roosevelt.edu/profile/sjamaliannas...   
2            https://www.roosevelt.edu/profile/mruth   
3        https://www.roosevelt.edu/profile/msalahieh   
4         https://www.roosevelt.edu/profile/awolpert   
5          https://www.roosevelt.edu/profile/rwright   

                                 Course                             Email  \
0  » \n    \n          Computer Science            edantsin@roosevelt.edu   
1  » \n    \n          Computer Science  sjamaliannasrabadi@roosevelt.edu   
2  » \n    \n          Co

**Remove extra spaces from the elements.**

In [19]:

#Function to remove extra space
def remove_extra_spaces(text):
    return ' '.join(text.split())

#Aply to college 
df_professors["Course"] = df_professors["Course"].apply(remove_extra_spaces)

print(df_professors)


               College                    Name  \
0  Rosevelt University          Evgeny Dantsin   
1  Rosevelt University  Saba Jamaliannasrabadi   
2  Rosevelt University            Michael Ruth   
3  Rosevelt University           Rami Salahieh   
4  Rosevelt University            Alex Wolpert   
5  Rosevelt University              Ray Wright   

                                                Link              Course  \
0         https://www.roosevelt.edu/profile/edantsin  » Computer Science   
1  https://www.roosevelt.edu/profile/sjamaliannas...  » Computer Science   
2            https://www.roosevelt.edu/profile/mruth  » Computer Science   
3        https://www.roosevelt.edu/profile/msalahieh  » Computer Science   
4         https://www.roosevelt.edu/profile/awolpert  » Computer Science   
5          https://www.roosevelt.edu/profile/rwright  » Computer Science   

                              Email  \
0            edantsin@roosevelt.edu   
1  sjamaliannasrabadi@roosevelt.edu   

**Saving in a DF for easy viewing**

In [14]:
# Salvar o DataFrame em um arquivo CSV
df_professors.to_csv("df_professors.csv", index=False)


In [20]:
# Create a Pandas DataFrame with the data of professors
df_professors = pd.DataFrame(data_professors)

# Save the DataFrame directly to an Excel sheet
with pd.ExcelWriter('df_professors.xlsx', engine='xlsxwriter') as writer:
    df_professors.to_excel(writer, sheet_name='Sheet1', index=False)
